In [22]:
from sub_func import *

In [7]:
# 오늘 날짜 (코스피 장이 열린 날만 한정, 예시)
today = '20230102'

# 조회할 정보의 기간
base_year = '2022'
base_quarter = 'Q4'
start_date = '20221001'
end_date = '20221231'

# 포트폴리오를 활용하는 실제 기간 (ex. 2022 Q4를 조회하여 2023 Q1을 위한 포트폴리오 생성)
target_year = '2023'
target_quarter = 'Q1'

json_file_path = './notion_page_ids.json'
data = read_json(json_file_path)

In [13]:
data

{'pf_selection_agent': {'2023_Q1_init_pf': '190cd049-9633-81d6-b95b-e61b981eafe0'},
 't_1': {'20230102_t_1_analyst_rp': '190cd049-9633-81c5-9ab8-d9a5ca711070',
  '20230102_t_1_portfolio_report': '190cd049-9633-8130-90f1-ccf933ca839d',
  '20230102_portfolio_weights': '190cd049-9633-819e-91fc-de471ab73c00',
  '20230102_t_1_trader_report': '190cd049-9633-8148-9d8e-fb98ccedc055'},
 't_2': {'20230102_t_2_t_4_trader_log': '190cd049-9633-8158-84aa-cb8358410ce6',
  '20230102_t_2_t_4_pf_update': '190cd049-9633-812b-a704-c7001c4b793a'},
 't_3': {},
 't_4': {},
 't_5': {}}

In [19]:
def gen_trader(today, base_year, base_quarter):
    data = read_json(json_file_path)

    t_1_trader_report = get_all_text_from_page(data['t_1'][f"{today}_t_1_trader_report"])
    t_2_t_4_trader_log = get_all_text_from_page(data['t_2'][f"{today}_t_2_t_4_trader_log"])

    system_prompt = """
        당신은 한국 주식 시장 트레이더입니다.
        이전 t1, t2, t4의 트레이더 output을 참조해서 당일 거래 성과 보고서를 작성하세요.
        당일 거래 내역을 리뷰하고 거래에 개선해야 할 점을 작성하세요.
    """

    prompt = f"""
        아래는 당일 주식시장 거래시간 전과 거래시간 때 생성된 주식 트레이더의 레포트와 로그입니다.
        레포트와 로그를 사용해 당일거래성과 보고서를 작성하세요.
        당일거래에 개선할 점 또한 보고서에 포함하세요. 개선할 점을 구체적으로 알려주세요.

        주식시장 거래시간 전 레포트:
        {t_1_trader_report}

        주식시장 거래시간 중 로그:
        {t_2_t_4_trader_log}

        보고서를 Markdown 형식으로 작성하세요.
        보고서는 반드시 주어진 정보에 대한 분석이 필요합니다.
        """

    response = to_GPT(system_prompt, prompt)
    trader_report = response["choices"][0]["message"]["content"]
    print("\nGenerated Trade Report:", trader_report)  # 최종 출력 확인

    to_DB('t_5', f"{today}_t_5_trader_report", f"{base_quarter}_{base_year}", str(trader_report))
    
    return trader_report

def gen_portfolio(today, base_year, base_quarter):
    data = read_json(json_file_path)

    t_1_analyst_report = get_all_text_from_page(data['t_1'][f"{today}_t_1_analyst_rp"])
    t_1_pf_manager_report = get_all_text_from_page(data['t_1'][f"{today}_t_1_portfolio_report"])
    t_5_trader_report = get_all_text_from_page(data['t_5'][f"{today}_t_5_trader_report"])

    system_prompt = """
        당신은 주식 포트폴리오를 분석하는 금용 포트폴리오 매니저입니다.
        주식시장 거래시간 이후, 포트폴리오 성과 및 리스크 분석을 하고 주식시장 거래시간 전 포트폴리오와 비교를 합니다. 
    """

    prompt = f"""
    주식시장 거래시간 이후, 포트폴리오 성과 및 리스크 분석을 하고 주식시장 거래시간 전 포트폴리오와 비교를 합니다. 
    주식시장 거래시간 이후 트레이터 레포트와 애널리스트 레포트와 주식시장 거래시간 전 애널리스트 레포트를 참조해 현재 포트폴리오 성과 및 리스크 분석을 합니다.
    주식시장 거래시간 전 포트폴리오와 비교를 해서 전략이 잘 들어맞았는지 평가를 합니다.
    만약, 잘 맞지 않았다면 이에 대한 개선방안을 제안합니다.
    또한, 레포트에 주식시장 거래시간 전 포트폴리오와 주식시장 거래시간 마감 후 변동된 포트폴리오를 기재합니다.

    --트레이더 레포트--
    {t_5_trader_report}

    
    --애널리스트 레포트--
    {t_1_analyst_report}


    --주식시장 거래시간 전 포트폴리오 레포트--
    {t_1_pf_manager_report}
    

    보고서를 Markdown 형식으로 작성하세요.
    보고서는 반드시 주어진 정보에 대한 분석이 필요합니다.
    """

    response = to_GPT(system_prompt, prompt)
    portfolio_report = response["choices"][0]["message"]["content"]
    print("\nGenerated Portfolio Report:", portfolio_report)

    to_DB('t_5', f"{today}_t_5_portfolio_report", f"{base_quarter}_{base_year}", str(portfolio_report))

    return portfolio_report

In [20]:
def t_5_main():
    gen_trader()
    gen_portfolio()

In [21]:
t_5_main()


Generated Trade Report: # 당일 거래 성과 보고서

## 1. 시장 종합 분석
- **시장 상황**: 당일 주식 시장은 전반적으로 하락세에서 시작되었으나, 과매도 상태에 있는 일부 종목에서 기술적 반등이 예상되었습니다.
- **거래량**: 일부 종목에서 거래량의 급증이 관찰되었으며, 이는 매수세가 강해질 가능성을 시사합니다.

## 2. 매매 성과
### 매수 및 매도 내역
- **매수**:
  - **031430**: 24,650원에 매수
  - **005820**: 20,650원에 매수
  - **007810**: 12,100원에 매수
  - **092220**: 1,837원에 매수
  - **120030**: 95,000원에 매수
  - **088790**: 2,630원에 매수
  - **001820**: 29,500원에 매수
  - **093050**: 15,850원에 매수
  - **011420**: 1,670원에 매수

- **매도**:
  - **004370**: 350,000원에 매도
  - **029460**: 17,050원에 매도
  - **092220**: 1,837원에 매도
  - **025560**: 1,391원에 매도
  - **003160**: 4,675원에 매도
  - **036570**: 439,500원에 매도
  - **000210**: 57,900원에 매도

### 포지션 별 성과
- **031430**: 매수 후 현재가 상승 기대.
- **005820**: 반등 가능성에 따른 매수 포지션.
- **007810**: 매수 가격 대비 양호한 위치.
- **092220**: 매도 후 손실 없음.
- **120030**: 과매도 상태에서 매수, 이후 반등 기대.

## 3. 거래에 대한 개선점
### 개선해야 할 점
1. **관망 시간의 최적화**: 오전 동안 관망만 하였던 종목들이 많아, 좀 더 적극적으로 기술적 반등을 노리는 매수 전략이 필요했습니다. 거래 시작 직후부터 적절한 매수 기회를 포착하는 것이 중

In [1]:
from t5 import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
today = '20230102'
base_year = '2022'
base_quarter = 'Q4'

t_5_main(today, base_year, base_quarter)


Generated Trade Report: # 당일 거래 성과 보고서

## 1. 시장 상황 요약
- **시장 동향**: 당일 시장은 전반적으로 하락세를 보였으며, 여러 종목에서 과매도 상태가 지속되었습니다. 리스크 관리가 중요한 날이었으며, 기술적 반등의 가능성을 염두에 두고 접근했습니다.
- **거래량**: 특정 종목에서 거래량의 급증이 관찰되었고, 이는 매수세가 강해질 수 있는 신호로 해석되었습니다.

## 2. 거래 내역
### 매수 및 매도 내역
- **031430**: 매수 24,650원 (1주), 매수 24,200원 (1주)
- **004370**: 매도 350,000원 (1주), 매도 348,500원 (1주)
- **005820**: 매수 20,650원 (1주), 매수 20,650원 (5주)
- **195870**: 매수 35,950원 (1주), 매수 35,600원 (1주)
- **007810**: 매수 12,100원 (1주)
- **092220**: 매수 1,837원 (10주), 매도 1,837원 (10주)
- **120030**: 매수 98,700원 (10주), 매수 95,000원 (1주)
- **001820**: 매수 29,500원 (10주), 매수 30,000원 (1주)

### 주요 성과
- **매도 성과**: 004370에서 달성한 매도 성과가 눈에 띄며, 시장의 하락세 속에서도 성공적인 청산이 이루어졌습니다.
- **신규 매수**: 과매도 상태에 있는 종목들에서 매수 포지션을 취함으로써 향후 반등에 기대를 걸 수 있는 포지션을 확보했습니다.

## 3. 분석 및 개선점
### 거래 성과 분석
- **매수 포지션**: 여러 종목에서 매수 포지션을 취했으나, 일부 종목에서는 명확한 목표가를 설정하지 않았거나 청산 시점을 놓친 경향이 있습니다. 특히 120030과 195870 종목에서 매수가 이루어졌지만, 명확한 청산 전략이 부족했습니다.
- **손절매 설정**: 손절매 가격을 설정하는 것이 중요함에도 불구하고, 일부 종목에서는 손절매를